In [ ]:
import fiftyone as fo
import fiftyone.utils.huggingface as fouh

# Load the dataset from Hugging Face if it's your first time using it

# dataset = fouh.load_from_hub(
# "Voxel51/Coursera_lecture_dataset_train", 
# dataset_name="lecture_dataset_train", 
# persistent=True)

In [ ]:
 #clone the dataset to avoid modifying the original dataset
cloned_dataset = fo.load_dataset("lecture_dataset_train_clone")

Finding Duplicate Objects
One common use case for IoU is identifying duplicate objects in your dataset. You can use the compute_max_ious function to calculate the maximum IoU between objects in the same image. This can help you find and remove duplicate bounding boxes, which can be detrimental to model performance.

`fiftyone.utils.iou.find_duplicates(sample_collection, label_field, iou_thresh=0.999, method='simple', progress=None, **kwargs)`